## Lecture 5.2: Molecular Evolution

In [5]:
import numpy as np
import math
import matplotlib.pyplot as plt
import random
from scipy.special import comb

****
<span style="color:red;">**Example: 5.6** K80</span>

**Consider the Kimura 1980 Model for molecular evolution:**

**2. Consider sequences with 10 bases, and simulate molecular evolution for $t=20$ time steps assuming $\alpha=0.1$ and $\beta=0.04$.** 

The K80 model describes evolution at every base, when considering multiple bases we assume evolution is independent at each.

We can imagine simulating sequences as placing mutations on a timeline from the ancestral sequence to the present day (t=20). The simulation itself can be carried out with a Gillespie simulation.  

*Step 0:* Initialization, simulate a random ancestral sequence from the stationary distribution.  Recall that the stationary distribution for this model is simply:
$$
\vec{\pi}=\left[\frac{1}{4},\frac{1}{4},\frac{1}{4},\frac{1}{4}\right]
$$

*Step 1:* Calculate the total rate at which mutations occur.  This calculation is relatively easy in the K80 model given that no matter what the base is the rate of mutating away from that base is $\alpha+2\beta$ such that the total mutation rate is: $n_{sites}*(\alpha+2\beta)$, in a more complex model this would require summing over the bases.  Let $X_i$ be the state of the $i^{th}$ base, then:

$$
\lambda_{Tot}=\sum_i -Q_{X_{i},X_{i}}
$$

Given the total mutation rate, simulate the time to the next event from the exponential distribution.  

Update the time

*Step 2:* Choose the event/mutation to have happened by drawing the base and then the type mutation relative to their respective probabilities. This is most efficiently done in two steps by first choosing a base: $\Pr(i)=\frac{-Q_{X_{i},X_{i}}}{\lambda_{Tot}}$ and then choosing a mutation type $\Pr(j)=\frac{Q_{X_{i},j}}{-Q_{X_{i},X_{i}}}$

Update the state

*Repeat Step 1 and 2 Until end time*

In [11]:
# Define the possible nucleotides
nucleotides = [0, 1, 2, 3] #(0:T, 1:C, 2: A, 3: G)

# Simulate a sequence of 20 letters with equal probability
sequence = random.choices(nucleotides, k=50)

# Print the generated sequence
print(np.array(sequence))


[1 2 3 3 1 1 4 2 3 3 3 3 3 4 1 1 3 4 4 4 2 3 1 3 2 1 1 1 2 4 4 4 1 2 4 3 1
 3 4 1 3 2 4 3 1 4 4 2 2 3]


In [144]:
def simJC69(alpha, beta, tMax):
    QMtrx=np.array([[-(alpha+2*beta), alpha, beta, beta],
                    [alpha,-(alpha+2*beta),beta,beta],
                    [beta,beta,-(alpha+2*beta),alpha],
                    [beta,beta,alpha,-(alpha+2*beta)]])
    ##Simulate a random sequence 
    nucleotides = [0, 1, 2, 3] #(0:T, 1:C, 2: A, 3: G)
    nBase=10
    seq = random.choices(nucleotides, k=nBase)
    t=0
    
    print(round(t,3))
    print(seq)
    # Calcualte the time until the first event
    rateTot=sum(-QMtrx[seq[i],seq[i]] for i in range(nBase))
    Deltat=np.random.exponential(scale=1/rateTot)
    while t+Deltat <= tMax:
        ## Update time
        t+=Deltat
        # Choose a focal base
        weight1=np.array([-QMtrx[seq[i],seq[i]]/rateTot for i in range(nBase)])
        base=np.random.choice(range(nBase), p=weight1)
        #Choose a nucleotide
        i=seq[base] #current nucleotide at the focal base
        altNeuc=np.array([j for j in range(4) if j != i])
        weight2=np.array([QMtrx[i,j] for j in range(4) if j != i])
        neuc=np.random.choice(altNeuc, p=weight2/np.sum(weight2))
        ## Update Sequence
        seq[base]=neuc
        print(round(t,3))
        print(seq)
        # Draw new Delta t
        rateTot=sum(-QMtrx[seq[i],seq[i]] for i in range(nBase))
        Deltat=np.random.exponential(scale=1/rateTot)
    return seq


In [145]:
temp=simJC69(0.1, 0.04, 10)

0
[3, 3, 1, 3, 1, 2, 1, 0, 3, 3]
0.032
[3, 2, 1, 3, 1, 2, 1, 0, 3, 3]
0.093
[3, 2, 1, 3, 1, 3, 1, 0, 3, 3]
0.341
[3, 2, 1, 3, 1, 3, 1, 1, 3, 3]
1.999
[3, 2, 1, 3, 1, 3, 1, 2, 3, 3]
2.198
[3, 3, 1, 3, 1, 3, 1, 2, 3, 3]
3.751
[3, 3, 1, 3, 1, 3, 1, 0, 3, 3]
4.408
[3, 3, 1, 3, 1, 3, 1, 3, 3, 3]
5.603
[3, 1, 1, 3, 1, 3, 1, 3, 3, 3]
5.996
[3, 0, 1, 3, 1, 3, 1, 3, 3, 3]
6.103
[3, 0, 1, 3, 0, 3, 1, 3, 3, 3]
6.258
[0, 0, 1, 3, 0, 3, 1, 3, 3, 3]
6.278
[0, 0, 1, 3, 0, 3, 1, 3, 2, 3]
6.667
[0, 0, 1, 3, 0, 3, 1, 3, 0, 3]
6.843
[0, 0, 1, 3, 0, 2, 1, 3, 0, 3]
7.077
[0, 0, 3, 3, 0, 2, 1, 3, 0, 3]
7.368
[0, 0, 3, 3, 0, 3, 1, 3, 0, 3]
7.836
[0, 0, 3, 3, 0, 3, 1, 2, 0, 3]
7.98
[0, 0, 3, 3, 0, 3, 1, 0, 0, 3]
8.201
[0, 0, 3, 3, 1, 3, 1, 0, 0, 3]
8.22
[0, 0, 3, 3, 1, 3, 1, 0, 0, 2]
8.23
[0, 0, 3, 3, 1, 3, 1, 2, 0, 2]
8.5
[0, 0, 3, 3, 1, 2, 1, 2, 0, 2]
8.705
[0, 0, 0, 3, 1, 2, 1, 2, 0, 2]
9.49
[3, 0, 0, 3, 1, 2, 1, 2, 0, 2]


**3. Do the observed frequencies of each nucleotide at the end of your simulation match your expectations from the stationary distribution?**

In [147]:
# Count occurrences of i from 0 to 3
counts = {i: temp.count(i) for i in range(4)}

# Print the counts
for i, count in counts.items():
    print(f"Freq of {i}: {count/len(temp)}")

Freq of 0: 0.3
Freq of 1: 0.2
Freq of 2: 0.3
Freq of 3: 0.2


The frequencies are close to 1/4 or what we would expect.